In [1]:
import tensorflow as tf
from tensorflow.python.framework import graph_util

In [2]:
# ProtocolBufferファイルの作成
def freeze_graph(model_folder):
    checkpoint = tf.train.get_checkpoint_state(model_folder)
    input_checkpoint = checkpoint.model_checkpoint_path
    
    output_graph = model_folder + "/frozen_model.pb"

    output_node_names = "input,output,output2"

    clear_devices = True
    
    # clear_devices=Trueで学習時のデバイスに依存しないようにする
    saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)

    graph = tf.get_default_graph()
    input_graph_def = graph.as_graph_def()

    with tf.Session() as sess:
        saver.restore(sess, input_checkpoint)

        output_graph_def = graph_util.convert_variables_to_constants(
            sess, 
            input_graph_def, 
            output_node_names.split(",") 
        ) 
        with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph." % len(output_graph_def.node))

In [ ]:
freeze_graph('./RoadSign-Alexnet-ckpt')

In [4]:
def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we can use again a convenient built-in function to import a graph_def into the 
    # current default Graph
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def, 
            input_map=None, 
            return_elements=None, 
            name="prefix", 
            op_dict=None, 
            producer_op_list=None
        )
    return graph

In [5]:
graph = load_graph('./RoadSign-Alexnet-ckpt/frozen_model.pb')

for op in graph.get_operations():
    print op.name,op.outputs

prefix/input [<tf.Tensor 'prefix/input:0' shape=(?, 150528) dtype=float32>]
prefix/keep_prob [<tf.Tensor 'prefix/keep_prob:0' shape=<unknown> dtype=float32>]
prefix/Reshape/shape [<tf.Tensor 'prefix/Reshape/shape:0' shape=(4,) dtype=int32>]
prefix/Reshape [<tf.Tensor 'prefix/Reshape:0' shape=(?, 224, 224, 3) dtype=float32>]
prefix/conv2d/kernel [<tf.Tensor 'prefix/conv2d/kernel:0' shape=(11, 11, 3, 96) dtype=float32>]
prefix/conv2d/kernel/read [<tf.Tensor 'prefix/conv2d/kernel/read:0' shape=(11, 11, 3, 96) dtype=float32>]
prefix/conv2d/bias [<tf.Tensor 'prefix/conv2d/bias:0' shape=(96,) dtype=float32>]
prefix/conv2d/bias/read [<tf.Tensor 'prefix/conv2d/bias/read:0' shape=(96,) dtype=float32>]
prefix/conv2d/convolution [<tf.Tensor 'prefix/conv2d/convolution:0' shape=(?, 56, 56, 96) dtype=float32>]
prefix/conv2d/BiasAdd [<tf.Tensor 'prefix/conv2d/BiasAdd:0' shape=(?, 56, 56, 96) dtype=float32>]
prefix/conv2d/Relu [<tf.Tensor 'prefix/conv2d/Relu:0' shape=(?, 56, 56, 96) dtype=float32>]
pr

In [6]:
import numpy as np
from PIL import Image

In [7]:
#　グラフのパラメータをpbファイルのパラメータで初期化
sess = tf.Session(graph=graph)

# 「止まれ」のサンプル画像
img = Image.open('./test_dataset_AlexNet/Stop-00196.jpg', "r")
input_data = np.frombuffer(np.array(img),dtype=np.uint8)
input_data = input_data.astype(np.float32)
input_data /= 255.0

predict = sess.run('prefix/output2:0',{'prefix/input:0':input_data.reshape(-1,224*224*3),'prefix/keep_prob:0':1.0})
print(predict)

Index = np.argmax(predict)
if(Index == 0):
    print "answer : Stop"
elif(Index == 1):
    print "answer : LimitSpeed10"
elif(Index == 2):
    print "answer : LimitSpeed20"
elif(Index == 2):
    print "answer : LimitSpeed20"

[[  9.99999404e-01   5.70434281e-07   5.25940314e-09   1.46564380e-12]]
answer : Stop
